In [25]:
import pandas as pd
import numpy as np
import pandas as pd
from surprise import SVD, evaluate
from surprise import KNNBaseline
from surprise.model_selection import cross_validate
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise import accuracy
from collections import defaultdict
import pickle

In [15]:
data = pd.read_csv("output2.csv",header = None)
data.columns = ['book_id','user_id','rating']
data = pd.DataFrame({'user_id' : data['user_id'],'book_id':data['book_id'],'rating':data['rating'] })

In [16]:
data.head()

,user_id,book_id,rating
0,446728d221c1343b92e1e4ff5545a843,B004DI7I3M,4.0
1,a03ee5913f066e0eb692efd97be1df70,B004DI7I3M,4.0
2,7f2f12c793747ffca608e638382ba160,B004DI7I3M,4.0
3,017aa38f7f917d932315a405c52d854a,B004DI7I3M,3.0
4,a703ab36472441bf32b23d3757fdd38b,B004DI7I3M,5.0


In [17]:
reader = Reader(rating_scale=(1, 5))
dataset = Dataset.load_from_df(data[['user_id','book_id','rating']], reader)

In [18]:
trainset, testset = train_test_split(dataset, test_size=.30)

In [20]:
# First train an SVD algorithm.
algo = SVD(n_factors = 100)
algo.fit(trainset)

In [21]:
# Than predict ratings for all pairs (u, i) that are NOT in the training set.
predictions = algo.test(testset)

In [22]:
accuracy.rmse(predictions)

RMSE: 0.8860


0.8859988162345689

In [27]:
pkl_filename = "pickle_model.pkl"  
with open(pkl_filename, 'wb') as file:  
    pickle.dump(algo, file)

# Load from file
with open(pkl_filename, 'rb') as file:  
    pickle_model = pickle.load(file)

# Calculate the accuracy score and predict target values
predictions = pickle_model.test(testset)  
  

In [28]:
def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [29]:
top_n = get_top_n(predictions, n=10)

In [30]:
top_n

defaultdict(list,
            {'5edc3ba4bd5a9912f58a1137fd835c30': [('B00DGZI9FO',
               3.6567094849488058),
              ('B008RD34VO', 3.6045535911536217),
              ('B0026NBZCQ', 3.5659811913453527),
              ('B00ZYTUKVG', 3.4836967869921316),
              ('B00F7V28VG', 3.4779830620276315),
              ('B002KYHZHA', 3.471604483098615),
              ('B00BXQCHXI', 3.466559656707555),
              ('B00H6SZ06I', 3.4453093111032715),
              ('B00KD019P2', 3.4440552319798647),
              ('B000SEI1EA', 3.3979368713043407)],
             'AHT85NS5LHGA4': [('0375831002', 4.668707480077214),
              ('0143124544', 4.642493791560046),
              ('1439102767', 4.5802954295190075),
              ('147674355X', 4.399988022598111),
              ('0615556310', 4.378698668674051),
              ('0399159347', 4.221120748722829),
              ('0803494149', 4.203728953505278),
              ('1612184669', 4.203528432459055),
              ('144235